קודים של הקדמה

In [1]:

import pandas as pd
import geopandas as gpd


In [2]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [3]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [4]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

להעלות את הטבלה של גורי

In [5]:
col=['sector', 'geometry', 'ele_stu', 'mid_stu', 'high_stu']

In [6]:
shcl=up_load_shp(r'W:\Data\Forecast\תחומי אחריות\יצירת מידע\בתי ספר\education.shp')[col] #לסנן רק את השכבות שצריך בלי גנים

העלת שכבת טאז

In [7]:
col=['Taz_num','main_secto','Muni_Heb','zonetype','geometry']

In [21]:
taz=up_load_shp(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\current\background_files\TAZ_V4_230518_Published.shp')#[col]

להחבר לגיאוגרפיה ולהוריד אזורים שמחוץ לתחום שלנו

In [9]:
taz_without_palestinian=taz.loc[taz['main_secto']!='Palestinian']

In [10]:
shcl_with_taz=gpd.sjoin(taz_without_palestinian,shcl).drop(columns='geometry')

לבדוק אל מול נתוני למ"ס מה המצב של המידע שגורי יצר

לסכום לפי אזור תנועה ומגזר

לשנות את השמות עמודות לפי איך שאני מודלים צריכים

In [11]:
old_col=['ele_stu','mid_stu','high_stu']

In [12]:
shcl_with_taz_arab=shcl_with_taz.loc[shcl_with_taz['sector']==1].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [13]:
new_col=['ARA1','ARA2','ARA3']

In [14]:
shcl_with_taz_arab.columns=new_col

In [15]:
shcl_with_taz_hardi=shcl_with_taz.loc[shcl_with_taz['sector']==2].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [16]:
new_col=['UOA1','UOA2','UOA3']

In [17]:
shcl_with_taz_hardi.columns=new_col

In [18]:
shcl_with_taz_Jewish=shcl_with_taz.loc[shcl_with_taz['sector']==3].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [19]:
new_col=['SEA1','SEA2','SEA3']

In [20]:
shcl_with_taz_Jewish.columns=new_col